In [ ]:
#educare

import os
import cv2
import numpy as np
from PIL import Image
import sqlite3
import openpyxl
from openpyxl.styles import Font
from tkinter import *
conn=sqlite3.connect("dhi.db")


global take_attendance

def take_attendance():
    
    wb=openpyxl.load_workbook("attendance.xlsx")
    sheet=wb.active
     
    
    FC=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    cap=cv2.VideoCapture(0)
    recognizer=cv2.face.LBPHFaceRecognizer_create()
    recognizer.read("training.yml")
    
    conn=sqlite3.connect("dhi.db")
    
    
    #intializing dictonary of student details to 0 and 1
    attendace_list=conn.execute("SELECT id,name from face_id").fetchall()
    print(attendace_list)
    for i in range(len(attendace_list)):
        
        #print(attendace_list[i][1])
        sheet.cell(row=i+2,column=1).value=attendace_list[i][0]
        sheet.cell(row=i+2,column=2).value=attendace_list[i][1]
        
        
    
    attendance={}
    student={}
    for i in attendace_list:
        attendance[i[0]]=0
        student[i[0]]=1
        
        
    font=cv2.FONT_HERSHEY_SIMPLEX
    
    
    def set_student_zero(student):
        for i in student:
            i=0
    
    i=0
    
    attendance_status="attendance not taken"
    while 1:
        
        ret,img=cap.read()
        gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces=FC.detectMultiScale(gray,1.3,5)
        for(x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            id,conf=recognizer.predict(gray[y:y+h,x:x+w])
         
            cv2.imshow("image",img)
            
            i=i+1
            student[id]=student[id]+1
            
            if i==50:
                
                student1=student.values()
                new_key=max(student,key=student.get)
                max_value=max(student1)
                print(max_value)
                if(max_value>30):
                    if(attendance[new_key]==0):
                        attendance[new_key]=attendance[new_key]+1
                    
                    name=conn.execute("SELECT id,name from face_id where id=?",(id,)).fetchall()
                    attendance_status=name[0][1]+' is present'
                    
                    print(attendance_status)
                    i=0
                else:
                    attendance_status="could not able to scan face"
                    i=0
                
                set_student_zero(student)
                    
            
                
            
            
            cv2.putText(img=img, text=attendance_status, org=(100,50), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255,64,64),thickness=1)
            
            cv2.imshow("image",img)
    
          
            #print(name[0][1])
        if cv2.waitKey(1)==ord('q'):
           print(attendance)
           break
      
     
    rows=2
    columns=3
    j=0
    a=1
    while(True):
        j=j+1
        
        cell_value=sheet.cell(row=rows,column=columns).value
        if cell_value==None:
            sheet.cell(row=rows,column=columns).value=attendance[j]
            a=0
            
        if rows==sheet.max_row:
            
            if a==0:
                break
            else:
                columns=columns+1
                rows=1
                j=0
        rows=rows+1
    
    wb.save("attendance.xlsx")
    wb.close()
    conn.commit()
    conn.close()
            
    cap.release()
    cv2.destroyAllWindows()


def face_train():

    recognizer=cv2.face.LBPHFaceRecognizer_create()
    
    path="dataset/"
    
    
    def getImagewithId(path):
        imagePath=[os.path.join(path, f) for f in os.listdir(path)]
        faces=[]
        IDs=[]
        for image in imagePath:
            facesImg=Image.open(image).convert('L')
            faceNP=np.array(facesImg,'uint8') #array format
            print(faceNP)
            ID=int(os.path.split(image)[-1].split(".")[1])   #user.1.1 etara eruthe so bari id tegibek so edu line
            print(ID)
            faces.append(faceNP)
            IDs.append(ID)
            cv2.imshow("adding faces for training ",faceNP)
            cv2.waitKey(10)
        return np.array(IDs),faces
    
    
    IDs,faces=getImagewithId(path)
    recognizer.train(faces,IDs)
    recognizer.save("training.yml")
    cv2.destroyAllWindows()
    
def face_detect():
    
    conn=sqlite3.connect("dhi.db")


    FC=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    cap=cv2.VideoCapture(0)
    recognizer=cv2.face.LBPHFaceRecognizer_create()
    global name
    recognizer.read("training.yml")
    id=0
    
    font=cv2.FONT_HERSHEY_SIMPLEX
    while 1:
        ret,img=cap.read()
        gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces=FC.detectMultiScale(gray,1.3,5)
        for(x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            id,conf=recognizer.predict(gray[y:y+h,x:x+w])
            cv2.imshow("image",img)
            
            name=conn.execute("SELECT id,name from face_id where id=?",(id,)).fetchall()
            
        
        if cv2.waitKey(1)==ord('q'):
            break
            
    
        
    cap.release()
    cv2.destroyAllWindows()
    
    def fetch_student_details(ID):
        
        fetched_details=conn.execute("SELECT * FROM student WHERE id=?",(ID,)).fetchall()
        print(fetched_details)
        print(fetched_details[0][0])
        
        fetch=Tk()
        fetch.geometry("1200x1200")
        fetch.attributes('-fullscreen',True)
        
        #title
        fetch.title("student detail")
        fetch_details_frame=LabelFrame(fetch,text='STUDENT DETAILS',highlightbackground = "white",bd=6,relief=GROOVE,fg='#05386B',bg='#76DC93',
                                       font=('times new roman',35,'bold'))
        fetch_details_frame.place(x=100,y=100,width=1700,height=900)
        #background
        fetch.configure(bg="#76DC93")
        
        
        def student_close():
            
            fetch.destroy()
        global Std_name1
        Label(fetch_details_frame,text="ID",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=50)
        student_id=Entry(fetch_details_frame,font=('times new roman',25),bd='5',fg='#05386B')
        student_id.place(x=400,y=50,width=400)
        student_id.insert(0, str(fetched_details[0][0]))
        student_id['state']=DISABLED
        
        
        Label(fetch_details_frame,text="NAME",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=150)
        
            
        
        student_name=Entry(fetch_details_frame,bd=5,bg="white",fg="#05386B",font=("times new roman",25))
        student_name.place(x=400,y=150,width=400)
        student_name.insert(0,str(fetched_details[0][1]))
        student_name['state']=DISABLED
        
        Label(fetch_details_frame,text="18**81",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=250)
        Label(fetch_details_frame,text="18**82",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=350)
        Label(fetch_details_frame,text="18**83",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=450)
        Label(fetch_details_frame,text="18**84",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=550)
        Label(fetch_details_frame,text="TOTAL",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=650)
        
        
        sub1=Entry(fetch_details_frame,bg="white",bd='5',fg="#05386B",font=("times new roman",25))
        sub1.place(x=400,y=250,width=400)
        sub1.insert(0,str(fetched_details[0][2]))
        sub1['state']=DISABLED
        
        sub2=Entry(fetch_details_frame,bg="white",bd='5',fg="#05386B",font=("times new roman",25))
        sub2.place(x=400,y=350,width=400)
        sub2.insert(0, str(fetched_details[0][3]))
        sub2['state']=DISABLED
        
        sub3=Entry(fetch_details_frame,bg="white",bd='5',fg="#05386B",font=("times new roman",25))
        sub3.place(x=400,y=450,width=400)
        sub3.insert(0, str(fetched_details[0][4]))
        sub3['state']=DISABLED
        
        sub4=Entry(fetch_details_frame,bg="white",bd='5',fg="#05386B",font=("times new roman",25))
        sub4.place(x=400,y=550,width=400)
        sub4.insert(0, str(fetched_details[0][5]))
        sub4['state']=DISABLED
        
        total=Entry(fetch_details_frame,bg="white",bd='5',fg="#05386B",font=("times new roman",25))
        total.place(x=400,y=650,width=400)
        total.insert(0, str(fetched_details[0][6]))
        total['state']=DISABLED
        
        
        
        Button(fetch_details_frame,text="BACK",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=student_close).place(x=100,y=750, width=200)
    
        
        fetch.mainloop()
            
        
    student=Tk()
    student.geometry("1200x1200")
    
    
    #title
    student.title("Student")
    student_frame=LabelFrame(student,text='STUDENT LAYOUT',highlightbackground = "white",bd=6,relief=GROOVE,fg='#05386B',bg='#76DC93',font=('times new roman',35,'bold'))
    student_frame.place(x=100,y=100,width=1000,height=800)
    #background
    student.configure(bg="#76DC93")
    Label(student_frame,text="NAME",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=100)
    
    student_capture_name=Entry(student_frame,bg="white",bd='5',fg="#05386B",font=("times new roman",25))
    student_capture_name.place(x=300,y=100)
    student_capture_name.insert(0,str(name[0][1]))
    student_capture_name['state']=DISABLED
    
    student_back=Button(student_frame,text="FETCH",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=lambda:fetch_student_details(id))
    student_back.place(x=300,y=200,width=200) 
    
    student_back=Button(student_frame,text="BACK",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=student.destroy)
    student_back.place(x=100,y=600,width=200)      
    
    #student_back1=Button(student,text="xcvbnm,",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=stu)
    #student_back1.place(x=290,y=350)
    student.mainloop()
    conn.commit()
    conn.close()
    
def face_read():
    add_student=Tk()
    conn=sqlite3.connect("dhi.db")
    add_student.geometry("1200x1200")
    add_student.title("add student")
    add_student_frame=LabelFrame(add_student,text='ADD STUDENT',highlightbackground = "white",bd=6,relief=GROOVE,fg='#05386B',bg='#76DC93',font=('times new roman',35,'bold'))
    add_student_frame.place(x=100,y=100,width=1000,height=800)
    
    
    def generate():
        
        
        
        conn.execute("INSERT INTO face_id(name) VALUES(?)",(add_student_name.get(),))
        generate_id['state']=DISABLED
     
        id_name=conn.execute("SELECT id FROM face_id").fetchall()
        
        add_student_id['state']=NORMAL
        add_student_id.insert(0,str(id_name[-1][0]))
        add_student_id['state']=DISABLED
        
    def student_close():
        add_student.destroy()
        
    def capture_student():
        FC=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
        cap=cv2.VideoCapture(0)
        add_student_id['state']=NORMAL
        ID=add_student_id.get()
        add_student_id['state']=DISABLED
        
        count=0
        while(1):
            ret,img=cap.read()
            gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces=FC.detectMultiScale(gray,1.3,5)
            for(x,y,w,h) in faces:
                count=count+1
                cv2.imwrite("dataset/User."+str(ID)+"."+str(count)+".jpg",gray[y:y+h,x:x+w])
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                cv2.waitKey(1)
            cv2.imshow("image",img)
            cv2.waitKey(1)
            if count>50:
                break
        
        conn.commit()
        conn.close()
        cap.release()
        cv2.destroyAllWindows()
        
    Label(add_student_frame,text="NAME",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=100)
    Label(add_student_frame,text="ID",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=200)
    Label(add_student_frame,text="FACE ID",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=300)
    
    
    add_student_name=Entry(add_student_frame,bg="white",bd='5',fg="#05386B",font=("times new roman",25))
    add_student_name.place(x=300,y=100)
    add_student_id=Entry(add_student_frame,bg="white",bd='5',fg="#05386B",state='disabled',font=("times new roman",25))
    add_student_id.place(x=300,y=200)
    
    generate_id=Button(add_student_frame,text="GENERATE ID",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=generate)
    generate_id.place(x=700,y=100, width=200)
    
    add_student_capture=Button(add_student_frame,text="CAPTURE",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=capture_student)
    add_student_capture.place(x=370,y=300, width=200)
    
    
    add_student_close=Button(add_student_frame,text="BACK",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=student_close)
    add_student_close.place(x=100,y=600, width=200)
    
    
    #background
    add_student.configure(bg="#76DC93")
    add_student.mainloop()
    conn.commit()
    conn.close()
        
        
    
    

def teacher_login1():
    global teacher_login1
    try:
        
        password1=conn.execute("SELECT password FROM login WHERE id=?",(user_id.get(),)).fetchall()
        
        if password.get()==password1[0][0]:
            
            def student_details():
                
                teacher=Tk()
                teacher.geometry("1200x1200")
                teacher.attributes('-fullscreen',True)
                
                #title
                teacher.title("Teacher")
                student_details_frame=LabelFrame(teacher,text='STUDENT DETAILS',highlightbackground = "white",bd=6,relief=GROOVE,fg='#05386B',
                                                 bg='#76DC93',font=('times new roman',35,'bold'))
                student_details_frame.place(x=100,y=100,width=1700,height=900)
                #background
                teacher.configure(bg="#76DC93")
                
                def marks_total():
                    
                    global total_marks
                    total_marks=int(sub1.get())+int(sub2.get())+int(sub3.get())+int(sub4.get())
                    total['state']=NORMAL
                    total.insert(0,str(total_marks))
                    total['state']=DISABLED
                    submit_button['state']= NORMAL
                    
                    
                    
                  
                def fetch():
                    std_name=conn.execute("SELECT name FROM face_id WHERE id=?",(student_id.get(),)).fetchall()
                    
                    std_name1=std_name[0][0]
                    student_name['state']=NORMAL
                    student_name.insert(0,str(std_name1))
                    student_name['state']=DISABLED
                    sub1['state']=NORMAL
                    sub2['state']=NORMAL
                    sub3['state']=NORMAL
                    sub4['state']=NORMAL
                def submit_details():
                    try:
                        conn.execute("INSERT INTO student(id,name,s1,s2,s3,s4,total) VALUES(?,?,?,?,?,?,?)",
                                     (student_id.get(),student_name.get(),sub1.get(),sub2.get(),sub3.get(),sub4.get(),total.get()))
                    except:
                        print("error")
                        
                def student_close():
                    
                    teacher.destroy()
                global Std_name1
                Label(student_details_frame,text="ID",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=50)
                student_id=Entry(student_details_frame,font=('times new roman',25),bd='5',fg='#05386B')
                student_id.place(x=400,y=50,width=400)
                
                Button(student_details_frame,text="FETCH",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=fetch).place(x=1200,y=50, width=200)
                Label(student_details_frame,text="NAME",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=150)
                std_name=conn.execute("SELECT name FROM face_id WHERE id=?",(student_id.get(),)).fetchall()
                    
                
                student_name=Entry(student_details_frame,state="disabled",bd=5,bg="white",fg="#05386B",font=("times new roman",25))
                student_name.place(x=400,y=150,width=400)
                
                Label(student_details_frame,text="18**81",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=250)
                Label(student_details_frame,text="18**82",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=350)
                Label(student_details_frame,text="18**83",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=450)
                Label(student_details_frame,text="18**84",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=550)
                Label(student_details_frame,text="TOTAL",bg="#76DC93",fg="#05386B",font=("times new roman",25)).place(x=100,y=650)
                
                
                sub1=Entry(student_details_frame,bg="white",state="disabled",bd='5',fg="#05386B",font=("times new roman",25))
                sub1.place(x=400,y=250,width=400)
                sub2=Entry(student_details_frame,bg="white",state="disabled",bd='5',fg="#05386B",font=("times new roman",25))
                sub2.place(x=400,y=350,width=400)
                sub3=Entry(student_details_frame,bg="white",state="disabled",bd='5',fg="#05386B",font=("times new roman",25))
                sub3.place(x=400,y=450,width=400)
                sub4=Entry(student_details_frame,bg="white",state="disabled",bd='5',fg="#05386B",font=("times new roman",25))
                sub4.place(x=400,y=550,width=400)
                total=Entry(student_details_frame,bg="white",state="disabled",bd='5',fg="#05386B",font=("times new roman",25))
                total.place(x=400,y=650,width=400)
                
                
                Button(student_details_frame,text="TOTAL",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=marks_total).place(x=1200,y=550, width=200)
                Button(student_details_frame,text="BACK",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=student_close).place(x=100,y=750, width=200)
                submit_button=Button(student_details_frame,text="SUBMIT",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),state="disabled",command=submit_details)
                submit_button.place(x=1200,y=650, width=200)
                
                
                teacher.mainloop()
                
            def t1_close():
                t1.destroy()
            
            global end
            def end():
                t1.destroy()
                root.destroy()
            
           # bt=Button(t1,text="Add Student Details",bg="green",fg="white",font=("Franklin Gothic Medium",18),command=details).place(x=480,y=380)
            global t1 
            t1=Tk()
            
            t1.geometry("1200x1200")
            t1.attributes('-fullscreen',True)
        
        #title
            t1.title("Teacher")
            teacher_frame=LabelFrame(t1,text='TEACHER LOGIN',highlightbackground = "white",bd=6,relief=GROOVE,fg='#05386B',bg='#76DC93',font=('times new roman',35,'bold'))
            teacher_frame.place(x=100,y=100,width=1700,height=900)
        
        #background
            t1.configure(bg="#76DC93")
            lab=Label(teacher_frame,text="CHOOSE OPERATION",bg="#05386B",fg="#76DC93",font=("times new roman",30,"bold"))
            lab.place(x=700,y=100)
            bt=Button(teacher_frame,text="ADD STUDENT DETAILS",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=student_details).place(x=150,y=200,width=400)
            bt=Button(teacher_frame,text="TAKE ATTENDANCE",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=take_attendance).place(x=1200,y=200,width=400)
            bt=Button(teacher_frame,text="ADD STUDENT",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=face_read).place(x=150,y=300,width=400)
            
            bt=Button(teacher_frame,text="BACK",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=t1_close).place(x=150,y=700,width=400)
            bt=Button(teacher_frame,text="EXIT",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=end).place(x=1200,y=700,width=400)
            
        else:
            print("error")
            t1.destroy()
        
    except:
        print("error")
        root.destroy()



def student_login():
    def stu():
        s1=Tk()
        s1.geometry("800x800")
    
    
    #title
        s1.title("Student Details")
        s1.configure(bg="#76DC93")
    
    #background
    
        lab=Label(s1,text="Stydents's Layout",bg="red",fg="white",font=("Arial Rounded MT Bold",25,"bold"))
        lab.place(x=250,y=80)
        
        lab=Label(s1,text="S_Name",bg="black",fg="white",font=("Arial Rounded MT Bold",19,"bold"))
        lab.place(x=100,y=180)
        
        tname=Label(s1,text="name",font=(14),bd='5')
        tname.place(x=330,y=180,width=200)
        
        lab=Label(s1,text="S_Attendance",bg="black",fg="white",font=("Arial Rounded MT Bold",19,"bold"))
        lab.place(x=100,y=260)
        
        tattendance=Label(s1,text="Attendance",font=(14),bd='5')
        tattendance.place(x=330,y=260,width=200)
        
        lab=Label(s1,text="Sub 1",bg="black",fg="white",font=("Arial Rounded MT Bold",19,"bold"))
        lab.place(x=100,y=340)
        
        t1=Label(s1,text="s1",font=(14),bd='5')
        t1.place(x=330,y=340,width=200)
        
        lab=Label(s1,text="Sub 2",bg="black",fg="white",font=("Arial Rounded MT Bold",19,"bold"))
        lab.place(x=100,y=420)
        
        t2=Label(s1,text="s2",font=(14),bd='5')
        t2.place(x=330,y=420,width=200)
        
        lab=Label(s1,text="Sub 3",bg="black",fg="white",font=("Arial Rounded MT Bold",19,"bold"))
        lab.place(x=100,y=500)
        
        t3=Label(s1,text="s3",font=(14),bd='5')
        t3.place(x=330,y=500,width=200)
        
        lab=Label(s1,text="Sub 4",bg="black",fg="white",font=("Arial Rounded MT Bold",19,"bold"))
        lab.place(x=100,y=580)
        
        t4=Label(s1,text="s4",font=(14),bd='5')
        t4.place(x=330,y=580,width=200)
        
        
        lab=Label(s1,text="Total",bg="black",fg="white",font=("Arial Rounded MT Bold",19,"bold"))
        lab.place(x=100,y=700)
        
        t4=Label(s1,text="tptal",font=(14),bd='5')
        t4.place(x=330,y=700,width=200)
        
        
        
    student=Tk()
    student.geometry("1000x1000")

#title
    student.title("Student")

#background
    student.configure(bg="#76DC93")
    lab=Label(student,text="Student's Layout",bg="red",fg="white",font=("Arial Rounded MT Bold",25,"bold"))
    lab.place(x=250,y=100)
            
    lab=Label(student,text="Name",bg="black",fg="white",font=("Arial Rounded MT Bold",19,"bold"))
    lab.place(x=290,y=250)
    
    bt=Button(student,text="Fetch Details",bg="green",fg="white",font=("Franklin Gothic Medium",18),command=stu).place(x=290,y=350)
    
    
    
def close():
    root.destroy()


root=Tk()

#window size
root.geometry("1900x1000")
root.attributes('-fullscreen',True)
#title
root.title("EDUCARE")

#background
root.configure(bg="#76DC93")
#bg = PhotoImage(file = "image.jpg")


lab=Label(root,text="EDUCARE",bg="#76DC93",fg="#05386B",font=("times new roman",60,"bold"))
lab.place(x=775,y=80)


#TEACHER LOGIN FRAME

teacher_login=LabelFrame(root,text='TEACHER LOGIN',highlightbackground = "white",bd=6,relief=GROOVE,fg='#05386B',bg='#76DC93',font=('times new roman',35,'bold'))
teacher_login.place(x=150,y=200,width=800,height=800)



lab=Label(teacher_login,text="ID",bg="#76DC93",fg="#05386B",font=("times new roman",25))
lab.place(x=100,y=100)

user_id=Entry(teacher_login,font=("times new roman",25),bd='5',fg='#05386B')
user_id.place(x=300,y=100,width=300)

lab=Label(teacher_login,text="PASSWORD",bg="#76DC93",fg="#05386B",font=("times new roman",25))
lab.place(x=100,y=200)

password=Entry(teacher_login,font=('times new roman',25),bd='5',show='*',fg='#05386B')
password.place(x=300,y=200,width=300)

bt=Button(teacher_login,text="LOGIN",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=teacher_login1).place(x=300,y=350,width=200)

#TEACHER LOGIN ENDS 

#STUDENT LOGIN
student_login=LabelFrame(root,text='STUDENT LOGIN',highlightbackground = "white",bd=6,relief=GROOVE,fg='#05386B',bg='#76DC93',font=('times new roman',35,'bold'))
student_login.place(x=1000,y=200,width=800,height=800)

student_lab=Label(student_login,text="STUDENT",bg="#76DC93",fg="#05386B",font=("times new roman",25))
student_lab.place(x=100,y=100)

#bt=Button(root,text="LOGIN",bg="red",fg="white",font=("Franklin Gothic Medium",18),command=student_login).place(x=480,y=500)

bt=Button(student_login,text="CAPTURE",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=face_detect).place(x=350,y=100, width=300)

close=Button(student_login,text="CLOSE",bg="#EDF5E1",fg="#05386B",font=("times new roman",18),command=close).place(x=350,y=600, width=400)

root.mainloop()